In [1]:
max_len = 100

embed_size = 256
encoder_num_banks = 16
decoder_num_banks = 8
num_highway_blocks = 4
reduction_factor = 5

learning_rate = 1e-4
batch_size = 32

from modules import (
    conv1d,
    normalize_in,
    highwaynet,
    gru,
    attention_decoder,
    prenet,
    embed,
    shift_by_one,
    conv1d_banks,
)
import tensorflow as tf

In [2]:
from utils import *
from sklearn.cross_validation import train_test_split
import time

/home/husein/.local/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
trainset = sklearn.datasets.load_files(container_path = 'data', encoding = 'UTF-8')
trainset.data, trainset.target = separate_dataset(trainset,1.0)
print (trainset.target_names)
print (len(trainset.data))
print (len(trainset.target))

['negative', 'positive']
10662
10662


In [4]:
concat = ' '.join(trainset.data).split()
vocabulary_size = len(list(set(concat)))
data, count, dictionary, rev_dictionary = build_dataset(concat, vocabulary_size)
print('vocab from size: %d'%(vocabulary_size))
print('Most common words', count[4:10])
print('Sample data', data[:10], [rev_dictionary[i] for i in data[:10]])

vocab from size: 20332
Most common words [('film', 1453), ('movie', 1270), ('one', 727), ('like', 721), ('story', 477), ('much', 386)]
Sample data [2723, 5347, 24, 35, 7006, 15, 64, 1951, 5348, 460] ['murderous', 'maids', 'may', 'well', 'comprehensive', 'films', 'also', 'strike', 'closest', 'truth']


In [ ]:
GO = dictionary['GO']
PAD = dictionary['PAD']
EOS = dictionary['EOS']
UNK = dictionary['UNK']

In [ ]:
def encode(inputs, is_training = True, scope = 'encoder', reuse = None):
    with tf.variable_scope(scope, reuse = reuse):
        prenet_out = prenet(inputs, scope = 'prenet', is_training = is_training)
        enc = conv1d_banks(
            prenet_out, K = encoder_num_banks, is_training = is_training
        )
        enc = tf.layers.max_pooling1d(enc, 2, 1, padding = 'same')
        enc = conv1d(enc, embed_size // 2, 3, scope = 'conv1d_1')
        enc = normalize_in(enc, activation_fn = tf.nn.relu)
        enc = conv1d(enc, embed_size // 2, 3, scope = 'conv1d_2')
        enc = normalize_in(enc, activation_fn = tf.nn.relu)
        enc += prenet_out
        for i in range(num_highway_blocks):
            enc = highwaynet(
                enc, units = embed_size // 2, scope = 'highwaynet_%d' % (i)
            )
        memory = gru(enc, embed_size // 2, True)
    return memory


def decode(
    inputs, memory, output_size, is_training = True, scope = 'decoder_layers', reuse = None
):
    with tf.variable_scope(scope, reuse = reuse):
        dec = prenet(inputs, is_training = is_training)
        dec = attention_decoder(dec, memory, embed_size)
        dec += gru(dec, embed_size, False, scope = 'gru1')
        dec += gru(dec, embed_size, False, scope = 'gru2')
        return tf.layers.dense(dec, output_size)

class Model:
    def __init__(self, dict_size, dimension_output):
        self.X = tf.placeholder(tf.int32, [None, None])
        self.Y = tf.placeholder(tf.int32, [None])
        self.is_training = tf.placeholder(tf.bool, None)
        
        encoder_embeddings = tf.Variable(tf.random_uniform([dict_size, embed_size], -1, 1))
        encoder_embedded = tf.nn.embedding_lookup(encoder_embeddings, self.X)
        
        self.decoder_inputs = embed(
            shift_by_one(self.X), dict_size, embed_size
        )
        with tf.variable_scope('net'):
            self.memory = encode(encoder_embedded, is_training = self.is_training)
            self.outputs = decode(
                self.decoder_inputs, self.memory, dimension_output, is_training = self.is_training
            )
        self.logits = self.outputs[:,-1]
        self.cost = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                logits = self.logits, labels = self.Y
            )
        )
        self.optimizer = tf.train.AdamOptimizer(
            learning_rate = learning_rate
        ).minimize(self.cost)
        correct_pred = tf.equal(
            tf.argmax(self.logits, 1, output_type = tf.int32), self.Y
        )
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [ ]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model(len(dictionary),len(trainset.target_names))
sess.run(tf.global_variables_initializer())

In [ ]:
maxlen = 100
batch_size = 16

In [ ]:
vectors = str_idx(trainset.data,dictionary,maxlen)
train_X, test_X, train_Y, test_Y = train_test_split(vectors, trainset.target,test_size = 0.2)

In [ ]:
from tqdm import tqdm
import time

EARLY_STOPPING, CURRENT_CHECKPOINT, CURRENT_ACC, EPOCH = 3, 0, 0, 0

while True:
    lasttime = time.time()
    if CURRENT_CHECKPOINT == EARLY_STOPPING:
        print('break epoch:%d\n' % (EPOCH))
        break

    train_acc, train_loss, test_acc, test_loss = 0, 0, 0, 0
    pbar = tqdm(
        range(0, len(train_X), batch_size), desc = 'train minibatch loop'
    )
    for i in pbar:
        batch_x = train_X[i : min(i + batch_size, train_X.shape[0])]
        batch_y = train_Y[i : min(i + batch_size, train_X.shape[0])]
        acc, cost, _ = sess.run(
            [model.accuracy, model.cost, model.optimizer],
            feed_dict = {
                model.Y: batch_y,
                model.X: batch_x
            },
        )
        assert not np.isnan(cost)
        train_loss += cost
        train_acc += acc
        pbar.set_postfix(cost = cost, accuracy = acc)

    pbar = tqdm(range(0, len(test_X), batch_size), desc = 'test minibatch loop')
    for i in pbar:
        batch_x = test_X[i : min(i + batch_size, test_X.shape[0])]
        batch_y = test_Y[i : min(i + batch_size, test_X.shape[0])]
        acc, cost = sess.run(
            [model.accuracy, model.cost],
            feed_dict = {
                model.Y: batch_y,
                model.X: batch_x
            },
        )
        test_loss += cost
        test_acc += acc
        pbar.set_postfix(cost = cost, accuracy = acc)

    train_loss /= len(train_X) / batch_size
    train_acc /= len(train_X) / batch_size
    test_loss /= len(test_X) / batch_size
    test_acc /= len(test_X) / batch_size

    if test_acc > CURRENT_ACC:
        print(
            'epoch: %d, pass acc: %f, current acc: %f'
            % (EPOCH, CURRENT_ACC, test_acc)
        )
        CURRENT_ACC = test_acc
        CURRENT_CHECKPOINT = 0
    else:
        CURRENT_CHECKPOINT += 1
        
    print('time taken:', time.time() - lasttime)
    print(
        'epoch: %d, training loss: %f, training acc: %f, valid loss: %f, valid acc: %f\n'
        % (EPOCH, train_loss, train_acc, test_loss, test_acc)
    )
    EPOCH += 1

train minibatch loop:   0%|          | 0/534 [00:00<?, ?it/s]

epoch: 0, pass acc: 0.000000, current acc: 0.735865
time taken: 371.9133610725403
epoch: 0, training loss: 0.655707, training acc: 0.584594, valid loss: 0.544825, valid acc: 0.735865



train minibatch loop:   0%|          | 0/534 [00:00<?, ?it/s]

epoch: 1, pass acc: 0.735865, current acc: 0.762588
time taken: 360.5551178455353
epoch: 1, training loss: 0.360291, training acc: 0.842537, valid loss: 0.586421, valid acc: 0.762588



train minibatch loop:   0%|          | 0/534 [00:00<?, ?it/s]

time taken: 358.11430191993713
epoch: 2, training loss: 0.170106, training acc: 0.933990, valid loss: 0.819403, valid acc: 0.751336



train minibatch loop:   0%|          | 0/534 [00:00<?, ?it/s]

time taken: 358.29570269584656
epoch: 3, training loss: 0.082946, training acc: 0.971509, valid loss: 1.063742, valid acc: 0.747586



test minibatch loop:  90%|█████████ | 121/134 [00:33<00:03,  3.56it/s, accuracy=0.812, cost=1.47] 

In [11]:
real_Y, predict_Y = [], []

pbar = tqdm(
    range(0, len(test_X), batch_size), desc = 'validation minibatch loop'
)
for i in pbar:
    batch_x = test_X[i : min(i + batch_size, test_X.shape[0])]
    batch_y = test_Y[i : min(i + batch_size, test_X.shape[0])]
    predict_Y += np.argmax(
        sess.run(
            model.logits, feed_dict = {model.X: batch_x, model.Y: batch_y}
        ),
        1,
    ).tolist()
    real_Y += batch_y

validation minibatch loop: 100%|██████████| 134/134 [01:05<00:00,  2.07it/s]


In [12]:
print(metrics.classification_report(real_Y, predict_Y, target_names = trainset.target_names))

             precision    recall  f1-score   support

   negative       0.73      0.77      0.75      1073
   positive       0.75      0.71      0.73      1060

avg / total       0.74      0.74      0.74      2133

